In [ ]:
# ============================================================
# === 🧠 IA Médicale — Détection du Diabète  ===
# ============================================================

!pip -q install imbalanced-learn xgboost shap gradio

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import shap
import gradio as gr
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, classification_report,
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score, confusion_matrix
)
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from google.colab import drive

RANDOM_STATE = 42
plt.style.use("seaborn-v0_8-whitegrid")

# === 1️⃣ Charger les données ===
def load_data():
    drive.mount("/content/drive", force_remount=True)
    path = "/content/drive/MyDrive/IA_Diabete/diabetes.csv"
    if not os.path.exists(path):
        raise FileNotFoundError(f"⚠️ Fichier introuvable : {path}")
    df = pd.read_csv(path)
    print("✅ Fichier chargé :", path)
    print("Shape :", df.shape)
    return df

# === 2️⃣ Nettoyage et features ===
def clean_and_engineer(df):
    df = df.copy()
    cols_zero = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
    for c in cols_zero:
        df[c] = df[c].replace(0, np.nan).fillna(df[c].median())
    df["Glucose_Insulin_ratio"] = df["Glucose"] / np.clip(df["Insulin"], 1, None)
    df["BMI2"] = df["BMI"] ** 2
    df["Glucose_BMI"] = df["Glucose"] * df["BMI"]
    df["Glucose_Age"] = df["Glucose"] / np.clip(df["Age"], 1, None)
    df["Pregnancies_Age"] = df["Pregnancies"] * df["Age"]
    return df

# === 3️⃣ Préparer les données ===
def prepare_xy(df):
    X = df.drop(columns=["Outcome"])
    y = df["Outcome"].astype(int)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
    )
    sm = SMOTE(random_state=RANDOM_STATE)
    X_train_b, y_train_b = sm.fit_resample(X_train, y_train)
    return X, y, X_train_b, X_test, y_train_b, y_test

# === 4️⃣ Entraînement optimisé ===
def tune_xgb(X_train, y_train):
    xgb = XGBClassifier(eval_metric="logloss", random_state=RANDOM_STATE, n_estimators=400)
    params = {
        "max_depth": [3, 4, 5],
        "learning_rate": [0.03, 0.05, 0.1],
        "subsample": [0.7, 0.9, 1.0],
        "colsample_bytree": [0.7, 0.9, 1.0],
        "min_child_weight": [1, 3, 5],
        "gamma": [0, 0.5, 1.0],
    }
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    pipe = Pipeline([("scaler", StandardScaler()), ("model", xgb)])
    rs = RandomizedSearchCV(pipe, param_distributions={
        "model__" + k: v for k, v in params.items()
    }, n_iter=15, cv=cv, scoring="roc_auc", n_jobs=-1, random_state=RANDOM_STATE)
    rs.fit(X_train, y_train)
    print(f"✅ Meilleur AUC (CV) : {rs.best_score_:.3f}")
    return rs.best_estimator_

# === 5️⃣ Évaluation du modèle ===
def evaluate(model, X_test, y_test):
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    print("\n🎯 Performance du modèle :")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("ROC AUC :", roc_auc_score(y_test, y_proba))
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    print("Matrice de confusion :\n", cm)

    # Courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_proba):.3f}")
    plt.plot([0, 1], [0, 1], "--", color="gray")
    plt.title("Courbe ROC")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    # Courbe Precision-Recall
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    plt.figure(figsize=(6,4))
    plt.plot(recall, precision, label=f"AP = {ap:.3f}")
    plt.title("Courbe Précision / Rappel")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.show()

# === 6️⃣ Sauvegarde du pipeline ===
def save_pipeline(pipe):
    path = "/content/drive/MyDrive/IA_Diabete/diabetes_pipeline_prof.pkl"
    joblib.dump(pipe, path)
    print("✅ Modèle sauvegardé :", path)

# === 7️⃣ Pipeline complet ===
def main():
    df = load_data()
    df = clean_and_engineer(df)
    X, y, X_train_b, X_test, y_train_b, y_test = prepare_xy(df)
    best_pipe = tune_xgb(X_train_b, y_train_b)
    evaluate(best_pipe, X_test, y_test)
    save_pipeline(best_pipe)
    return best_pipe, X_test

best_pipe, X_test = main()

# ========= FONCTION DE PRÉDICTION CORRIGÉE =========
def predict_diabetes(Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI,
                     DiabetesPedigreeFunction, Age):
    """Prédiction avec analyse SHAP fonctionnelle"""

    # Création du dataframe patient
    df_patient = pd.DataFrame([{
        "Pregnancies": Pregnancies,
        "Glucose": Glucose,
        "BloodPressure": BloodPressure,
        "SkinThickness": SkinThickness,
        "Insulin": Insulin,
        "BMI": BMI,
        "DiabetesPedigreeFunction": DiabetesPedigreeFunction,
        "Age": Age,
        "Glucose_Insulin_ratio": Glucose / max(Insulin, 1),
        "BMI2": BMI ** 2,
        "Glucose_BMI": Glucose * BMI,
        "Glucose_Age": Glucose / max(Age, 1),
        "Pregnancies_Age": Pregnancies * Age
    }])[X_test.columns]

    # Prédiction
    scaler = best_pipe.named_steps["scaler"]
    model = best_pipe.named_steps["model"]
    X_scaled = scaler.transform(df_patient)
    y_prob = float(model.predict_proba(X_scaled)[0, 1])
    y_pred = int(y_prob >= 0.5)

    # Rapport médical
    verdict = "🩺 **Diabétique**" if y_pred == 1 else "✅ **Non diabétique**"
    niveau = "élevé" if y_prob > 0.7 else ("modéré" if y_prob > 0.4 else "faible")
    couleur = "🔴" if y_prob > 0.7 else ("🟠" if y_prob > 0.4 else "🟢")

    markdown = f"""
## {couleur} Diagnostic IA : {verdict}

**Probabilité de diabète :** {y_prob*100:.1f}%
**Niveau de risque :** {niveau}

### 📋 Résumé des paramètres :
- **Glycémie** : {Glucose} mg/dL
- **IMC** : {BMI:.1f} kg/m²
- **Âge** : {Age} ans
- **Insuline** : {Insulin} µU/mL
"""

    # ========= GÉNÉRATION DU GRAPHIQUE SHAP =========
    try:
        explainer = shap.Explainer(model)
        shap_values = explainer(X_scaled)

        # Création du dataframe pour les impacts
        shap_df = pd.DataFrame({
            "Feature": X_test.columns,
            "Impact": shap_values.values[0]
        }).sort_values("Impact", key=abs, ascending=False).head(10)

        # Création du graphique
        plt.figure(figsize=(8, 5))
        colors = ["#00bfa6" if v > 0 else "#ff6b6b" for v in shap_df["Impact"]]
        plt.barh(shap_df["Feature"], shap_df["Impact"], color=colors)
        plt.title("🧠 Analyse SHAP — Impact des variables (Leith Chqoubi)", fontsize=13, pad=10)
        plt.xlabel("Influence sur le risque de diabète", fontsize=11)
        plt.gca().invert_yaxis()
        plt.tight_layout()

        # Sauvegarde du graphique
        shap_path = "/tmp/shap_patient.png"
        plt.savefig(shap_path, bbox_inches="tight", dpi=120)
        plt.close()

        return markdown, round(y_prob * 100, 2), shap_path

    except Exception as e:
        print(f"❌ Erreur SHAP : {e}")
        return markdown + f"\n\n⚠️ Erreur lors de la génération SHAP : {e}", round(y_prob * 100, 2), None

# ========= FONCTION SHAP GLOBALE =========
def global_shap_summary():
    """Génère l'analyse SHAP globale du modèle"""
    try:
        X_sample = X_test.sample(min(200, len(X_test)), random_state=42)
        xgb_model = best_pipe.named_steps["model"]
        scaler = best_pipe.named_steps["scaler"]
        X_scaled = scaler.transform(X_sample)

        explainer = shap.Explainer(xgb_model)
        shap_values = explainer(X_scaled)

        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values.values, X_sample, show=False)
        plt.title("📊 Analyse SHAP globale du modèle XGBoost", fontsize=14, pad=15)

        shap_global_path = "/tmp/shap_global.png"
        plt.savefig(shap_global_path, bbox_inches="tight", dpi=120)
        plt.close()

        return shap_global_path

    except Exception as e:
        print(f"❌ Erreur SHAP global : {e}")
        return None

# ========= INTERFACE GRADIO =========
with gr.Blocks(theme=gr.themes.Soft(primary_hue="teal", secondary_hue="gray")) as app:
    gr.HTML("""
    <div style='text-align:center; margin-bottom:20px'>
        <h1>🧠 IA Médicale — Détection du Diabète</h1>
        <h3>Analyse intelligente développée par <b style='color:#00bfa6'>Leith Chqoubi</b></h3>
        <p>Modèle XGBoost optimisé (SMOTE + standardisation + SHAP explainer)</p>
        <hr style='margin:10px 0'>
    </div>
    """)

    with gr.Tab("🔍 Prédiction IA"):
        with gr.Row():
            with gr.Column(scale=1):
                Pregnancies = gr.Slider(0, 15, 2, step=1, label="Grossesses (Pregnancies)")
                Glucose = gr.Slider(50, 220, 120, step=1, label="Glycémie (mg/dL)")
                BloodPressure = gr.Slider(40, 120, 70, step=1, label="Pression artérielle (mmHg)")
                SkinThickness = gr.Slider(5, 80, 25, step=1, label="Épaisseur de peau (mm)")
                Insulin = gr.Slider(0, 400, 80, step=1, label="Insuline (µU/mL)")
                BMI = gr.Slider(15.0, 60.0, 28.5, step=0.1, label="IMC (kg/m²)")
                DPF = gr.Slider(0.0, 2.5, 0.5, step=0.01, label="Facteur héréditaire (DPF)")
                Age = gr.Slider(18, 90, 35, step=1, label="Âge")
                btn = gr.Button("⚡ Lancer l'analyse IA", variant="primary")

            with gr.Column(scale=1):
                out_text = gr.Markdown(label="🧩 Rapport médical IA")
                out_prob = gr.Number(label="Probabilité (%)", precision=2)
                out_img = gr.Image(label="Explication SHAP (impact des variables)")

        btn.click(
            predict_diabetes,
            inputs=[Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DPF, Age],
            outputs=[out_text, out_prob, out_img]
        )

    with gr.Tab("📊 Analyse Interprétable SHAP"):
        gr.Markdown("""
        ### 🌐 Analyse globale du modèle XGBoost
        Ce graphique montre **l'influence moyenne de chaque variable** sur la prédiction du diabète.
        Les points rouges indiquent des valeurs élevées, les bleus des valeurs faibles.
        """)

        shap_btn = gr.Button("📈 Générer l'analyse SHAP complète", variant="primary")
        shap_img = gr.Image(label="SHAP Summary Plot (vue globale du modèle)")

        shap_btn.click(fn=global_shap_summary, inputs=[], outputs=[shap_img])

    gr.HTML("""
    <hr>
    <div style='text-align:center; font-size:14px; color:gray'>
        Développé avec 💡 Intelligence Artificielle — <b>Leith Chqoubi</b><br>
        <i>Prototype académique de diagnostic IA médicale (XGBoost + SHAP)</i>
    </div>
    """)

app.launch(share=True)

# Nouvelle section

# Nouvelle section